# Unstructured Spherical Voronoi Grids and Applications in Atmospheric Dynamics on the Sphere

---



## Introduction and motivation

The two dominant grids currently used in modern atmospheric models are the ones based on Voronoi grids and the ones based on Cubed Sphere grids.

Here, we will look into the construction of Spherical Voronoi grids using MPAS-Tools and Jigsaw software.

👉 This tutorial is best done offline, not in Google Collaboratory. We will assume you have installed some version of Linux and know a bit about command line and installation on such systems. This tutorial has been tested on Debian/Ubuntu environments.

👉 In case you only have access to Windows, it may be possible to run such software and tutorial, but, unfortunately, you will have to search on how to install such software on Windows, and check if it is possible.




---



## Software Install 


## Initial setups (see examples of the steps bellow)


1. Install an environment tool (Conda used here) - https://docs.conda.io/projects/miniconda/en/latest/
2. Install MPAS-TOOLS - http://mpas-dev.github.io/MPAS-Tools/stable/mesh_creation.html#spherical-meshes
3. Install Jigsaw - https://github.com/dengwirda/jigsaw-python/, see also https://github.com/dengwirda/jigsaw-geo-python/

Doing this in Google Colab is possible, but very slow. I recommend setting up the environment locally, using the commands commented below, and then simply running the next steps of the notebook.

1. Install Conda evniroment

If using Google Colab, uncomment the code in the cell below and run it.

In [ ]:
# --- Google Colab ---
# If you want to try to install the libraries in Colab, there goes a recipe
#   ... but this is very slow, usually.

# ---- Install Conda enviroment
#  : Colab ---
#!pip install condacolab
#import condacolab
#condacolab.install()

If not using Google Colab, install Conda from a terminal with the following commands:

```bash
    wget https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
    bash Miniconda3-latest-Linux-x86_64.sh
    source ~/miniconda3/bin/activate
```

Check installation with:
```bash
    conda --version
```

And add the conda-forge channel to install packages:
```bash
    conda config --add channels conda-forge
```


2. Install MPAS-Tools

Install MPAS-Tools in its own environment with the following commands:

```bash
    conda create --name mpas-tools python=3.11

    conda activate mpas-tools
    conda install -c conda-forge mpas_tools
```

Also install the jupyter notebook and the ipython kernel with

```bash
    conda install -c conda-forge jupyter
```

3. Install Jigsaw

With the mpas-tools environment activated (`conda activate mpas-tools`), install jigsaw with the following command:

```bash
    conda install jigsaw
    conda install jigsawpy
```

-----------------------------

## Setup for Egeon

Log into Egeon

```bash
ssh -Y $USER@egeon-login.cptec.inpe.br
```
entering in ``$USER`` the username you received from the organization team. You will then be prompted to give your password, which you should have received as well.

Load the enviroment

```bash
module load anaconda3-2022.05-gcc-11.2.0-q74p53i
conda init
source ~/.bashrc
conda config --add envs_dirs /pesq/share/monan/curso_OMM_INPE_2025/.conda/envs
conda activate vtx_env
```

Download notebook to your home/working directory

```bash
wget https://raw.githubusercontent.com/CGFD-USP/MPAS-References/master/WMO-Workshop2025/JigsawGrids_tutorial.ipynb
```



-----

### Open the notebook

At this point, we are assuming you are in a Conda environment called "mpas-tools" with jigsaw installed.

#### For local work with the notebook

Within this environment, you can open the Jypyter Notebook using either

```bash
    jupyter-lab JigsawGrids_tutorial.ipynb.ipynb
```

or

```bash
    jupyter-notebook JigsawGrids_tutorial.ipynb
```

Another (very good) option is to use VSCode to run and edit the notebook. In this case, just make sure you load your MPAS-Tools enviroment when oppening VSCode.


#### For remote login - with ssh to access the notebook

**Remote machine:** Within this environment, inside the server (Egeon), you can open the Jypyter Notebook using 
```bash
    jupyter-lab --no-browser
```

Note down the output information, for example
```bash
    http://localhost:8889/lab?token=6f9f724c39a9002fa78b303a12f4224a26ab70792c437a2a
```

**Local machine:** Now on your local machine, access the server with the port access:
```bash
    ssh -L 8889:localhost:8889 nome.sobrenome@150.163.212.73
```
Change the port, user and IP for the remote server.

Now open a browser and access
```bash
   http://localhost:8889
```

It will ask for a token, type the token that you saved when before, for instance: ``6f9f724c39a9002fa78b303a12f4224a26ab70792c437a2a``

Now go ahead with the notebook!!

In [ ]:
# Load libraries
import numpy as np
import argparse
import os

import xarray

import jigsawpy as jig
#import jigsaw_util as jutil

from mpas_tools.mesh.creation.jigsaw_to_netcdf import jigsaw_to_netcdf
from mpas_tools.mesh.conversion import convert
from mpas_tools.io import write_netcdf

import subprocess

import matplotlib.pyplot as plt

If the cell above did not run successfully, it means your installation is not done properly. Check the installation manuals of MPAS-TOOLS and JIGSAW.



---





## Basic Icosahedron grid

We start by defining some parameters of our grid. In this approach, it is convenient to define a different grid name for each different grid to be generated. We will create a folder with this name, and save many files related to grid in this folder.

In [ ]:
# Set a basename for output of grid files
output = "grid_icos"

out_dir=os.path.abspath(output)
out_base=os.path.basename(output)
out_basepath=out_dir+"/"+out_base
out_filename=out_dir+"/"+out_base+"_mpas.nc"

if not os.path.exists(out_dir):
    os.makedirs(out_dir)
else:
    print("Base dir already exists: ", out_dir)

print(out_dir)
print(out_basepath)
print(out_filename)

Check if the directory was correctly created, in the place you wanted. Adjust the commands above if you want to save the grids somewhere else, such as in a MPAS model grids folder.

We will start creating an icosahedron grid with Jigsaw, as illustration. The main call to Jigsaw is the following.

In [ ]:
#
#  Generated Icosahedral grid with JIGSAW
#
def jigsaw_gen_icos_grid(basename="mesh", level=4):

    # Main configuration JIGSAW
    # https://github.com/dengwirda/jigsaw-python/blob/master/jigsawpy/jig_t.py
    opts = jig.jigsaw_jig_t()
    #https://github.com/dengwirda/jigsaw-python/blob/master/jigsawpy/msh_t.py
    geom = jig.jigsaw_msh_t()

    # Filenames to be used
    opts.geom_file = basename+'.msh'
    opts.jcfg_file = basename+'.jig'
    opts.mesh_file = basename+'-MESH.msh'

    # Define geometry of mesh
    geom.mshID = "ellipsoid-mesh" #required to create spherical grids (you can also generate planar grid, or 3D grids)
    geom.radii = np.full(3, 1.000E+000, dtype=geom.REALS_t) #    MESH.RADII - [ 3x 1] array of principal ellipsoid radii.


    # save a JIGSAW MSH object to file.
    jig.savemsh(opts.geom_file, geom)

    # Grid generation options
    opts.hfun_hmax = +1.          # mesh-size function value.
    opts.mesh_dims = +2           # 2-dim. simplexes (2d spherical grid - surface)

    jig.cmd.icosahedron(opts, level, geom)

    return opts.mesh_file


Let's create the Icosahedron grid!

In [ ]:

# Grid options

#Icosahedral grid with 2 levels of refinement
mesh_file = jigsaw_gen_icos_grid(basename=out_basepath, level=2)

#Convert jigsaw mesh to netcdf
jigsaw_to_netcdf(msh_filename=mesh_file, output_name=out_basepath+'_triangles.nc', on_sphere=True,
                     sphere_radius=1.0)

#convert to mpas grid specific format
write_netcdf( convert(xarray.open_dataset(out_basepath+'_triangles.nc'),
        dir=out_dir, graphInfoFileName=out_basepath+"_graph.info"),
        out_filename)


The grid was created! Now we need to understand what was created and how we can plot it and use it!

In [ ]:
#Grid info
file = open(out_basepath+"_graph.info", 'r')

#1st line
ginfo = file.readline().split(" ")
nodes = ginfo[0]
edges = ginfo[1]
print("Nodes:", nodes)
print("Edges:", edges)
print()
print("How each node connects to neighbours")
gl1 = file.readline().rstrip()
print("This means that node #1 is connected to nodes ", gl1)

This is basically what defines the connectivity of a grid!

In [ ]:
#Grid points
file = open(out_basepath+"-MESH-ITER.msh", 'r')

#First few lines
gpoints = file.readlines(200)

for l in gpoints:
    print(l)


Points are in given in 3D cartesian coordinates (x,y,z). First 2 points are the South and North Pole.

The other files in the grid folder have move grid information in other formats. Take a look!

-----


## Grid Visualization

To visualize the grids we need auxiliar geometric constructions, which will be based on "patches".

In [ ]:
# Set a basename for existing grid files to plot
output = "grid_icos"

out_dir=os.path.abspath(output)
out_base=os.path.basename(output)
out_basepath=out_dir+"/"+out_base
out_filename=out_dir+"/"+out_base+"_mpas.nc"

print(out_dir)
print(out_basepath)
print(out_filename)

We will need to install a few packages for visualization:

1. Install Basemap (https://pypi.org/project/basemap/)

With the mpas-tools environment activated (`conda activate mpas-tools`), install basemap with the following command:

```bash
    conda install basemap
```

2. NETCDF4 should already be installed with jigsaw, if not, you can install it by hand

```bash
    conda install netcdf4
```


In [ ]:
import os
import sys
import pickle as pkle

import numpy as np
from netCDF4 import Dataset  # pip install netCDF4

import matplotlib.pyplot as plt

import matplotlib.collections as mplcollections
import matplotlib.patches as patches
import matplotlib.path as path
import matplotlib.cm as cm
from mpl_toolkits.basemap import Basemap  # pip install basemap

#----------------------------------------------------------------
# Module to create patches for MPAS grids
# Base on mpas_patches.py https://github.com/lmadaus/mpas_python, https://github.com/mgduda/MPAS-Plotting/blob/master/mpas-patches/mpas_patches.py
# Updated by P. Peixoto on Oct 2023 to allow other geometric features to be ploted
# -----------------------------------------------------------------

def update_progress(job_title, progress):
    length = 40
    block = int(round(length*progress))
    msg = "\r{0}: [{1}] {2}%".format(job_title, "#"*block + "-"*(length-block),
    round(progress*100, 2))
    if progress >= 1: msg += " DONE\r\n"
    sys.stdout.write(msg)
    sys.stdout.flush()

def get_mpas_patches(mesh, type="cell", msh_file="grid.nc", pickleFile=None):

    nCells = len(mesh.dimensions['nCells'])
    #nEdges = len(mesh.dimensions['nEdges'])
    nVertices = len(mesh.dimensions['nVertices'])
    nEdgesOnCell = mesh.variables['nEdgesOnCell']
    #verticesOnEdge = mesh.variables['verticesOnEdge']
    verticesOnCell = mesh.variables['verticesOnCell']
    #cellsOnCell = mesh.variables['cellsOnCell']
    cellsOnVertex = mesh.variables['cellsOnVertex']
    #cellsOnEdge = mesh.variables['cellsOnEdge']
    #latVertex = mesh.variables['latVertex']
    #lonVertex = mesh.variables['lonVertex']
    #latCell = mesh.variables['latCell']
    #lonCell = mesh.variables['lonCell']
    #latEdge = mesh.variables['latEdge']
    #lonEdge = mesh.variables['lonEdge']

    base_name = os.path.splitext(msh_file)[0]

    if (type=="triangle" or type=="dual"):
        patches_filename = base_name + '.tri.pat'
        nPat = nVertices
    else:
        patches_filename = base_name + '.cell.pat'
        nPat = nCells

    if pickleFile:
        pickle_fname = pickleFile
    else:
        pickle_fname = patches_filename

    mesh_patches = [None] * nPat

    print("Patches file:", pickle_fname)

    if(os.path.isfile(pickle_fname)):
        pickled_patches = open(pickle_fname,'rb')
        patch_collection = pkle.load(pickled_patches)
        pickled_patches.close()
        print("Pickle file (", pickle_fname, ") loaded succesfully")
        return patch_collection

    print("\nCreating patches...")
    print("If this is a large mesh, then this proccess will take a while...")

    for i in range(nPat):
        # For each cell/triangle, get the latitude and longitude points of its vertices
        # and make a patch of that point vertices

        if (type=="triangle" or type=="dual"):
            vertices = cellsOnVertex[i,:]
            vertices = np.append(vertices, vertices[0:1]) #Close the loop
            vertices -= 1 # The indexing needs to start in zero, not 1. (??)
            vert_lats = mesh.variables['latCell'][vertices]* (180 / np.pi)
            vert_lons = mesh.variables['lonCell'][vertices]* (180 / np.pi)
        else:
            vertices = verticesOnCell[i,:nEdgesOnCell[i]]
            vertices = np.append(vertices, vertices[0:1]) #Close the loop
            vertices -= 1 # The indexing needs to start in zero, not 1.
            vert_lats = mesh.variables['latVertex'][vertices]* (180 / np.pi)
            vert_lons = mesh.variables['lonVertex'][vertices]* (180 / np.pi)

        # Normalize longitude
        diff = np.subtract(vert_lons, vert_lons[0])
        vert_lons[diff > 180.0] = vert_lons[diff > 180.] - 360.
        vert_lons[diff < -180.0] = vert_lons[diff < -180.] + 360.

        coords = np.vstack((vert_lons, vert_lats))

        cell_path = np.ones(vertices.shape) * path.Path.LINETO
        cell_path[0] = path.Path.MOVETO
        cell_path[-1] = path.Path.CLOSEPOLY
        cell_patch = path.Path(coords.T,
                            codes=cell_path,
                            closed=True,
                            readonly=True)

        mesh_patches[i] = patches.PathPatch(cell_patch)

        update_progress("Creating Patch file: "+pickle_fname, i/nPat)

    print("\n")

    # Create patch collection
    patch_collection = mplcollections.PatchCollection(mesh_patches)

    # Pickle the patch collection
    pickle_file = open(pickle_fname, 'wb')
    pkle.dump(patch_collection, pickle_file)
    pickle_file.close()

    print("\nCreated a patch file for mesh: ", pickle_file)
    return patch_collection




In [ ]:
def plot_geometry(variable, file):

    # Open the NetCDF file and pull out the var at the given levels.
    # Check to see if the mesh contains the variable
    if not os.path.isfile(file):
        print("That file was not found :(")
        exit(-1)

    # Open the mesh using NetCDF4 Dataset.
    mesh = Dataset(os.path.join(file), 'r')
    print(mesh)
    print(mesh.variables.keys())
    lat_var = "lat"+variable
    lon_var = "lon"+variable
    # Check to see the variable is in the mesh
    if lat_var not in mesh.variables.keys():
        print("That variable was not found in this mpas mesh!", lat_var)
        print(mesh.variables.keys())
        sys.exit(-1)

    if lon_var not in mesh.variables.keys():
        print("That variable was not found in this mpas mesh!", lon_var)
        print(mesh.variables.keys())
        sys.exit(-1)

    # Pull the variable out of the mesh. Now we can manipulate it any way we choose
    # do some 'post-processing' or other meteorological stuff

    varlats = mesh.variables['lat'+variable][:] * (180 / np.pi)
    varlons = mesh.variables['lon'+variable][:] * (180 / np.pi)

    # Normalize latitude and longitude
    #lons[lons > 180.0] = lons[lons > 180.] - 360.
    #lons[lons < -180.0] = lons[lons < -180.] + 360.

    # Create or get the patch file for our current mesh
    patchtype="triangle"
    patch_collection_triangle = get_mpas_patches(mesh, type=patchtype, msh_file=file, pickleFile=None)

    patchtype="cell"
    patch_collection_cell = get_mpas_patches(mesh, type=patchtype, msh_file=file, pickleFile=None)

    %matplotlib inline

    print("Creating a plot ")

    fig = plt.figure(figsize=(18, 9), dpi=800)
    ax = plt.gca()

    # Initalize Basemap
    bmap = Basemap(projection='cyl',
                llcrnrlat=-90,
                urcrnrlat=90,
                llcrnrlon=0,
                urcrnrlon=360,
                resolution='l')

    color_map = cm.gist_ncar
    style = 'ggplot'


    bmap.drawcoastlines()

    #bmap.drawparallels(range(-90, 90, 30), linewidth=1, labels=[1,0,0,0], color='b')
    #bmap.drawmeridians(range(-180, 180, 45), linewidth=1, labels=[0,0,0,1], color='b', rotation=45)

    #patch_collection_triangle.set_array(lat[:])
    patch_collection_triangle.set_linewidths(0.2)
    patch_collection_triangle.set_linestyle("-")
    patch_collection_triangle.set_edgecolors("blue")         # No Edge Colors
    patch_collection_triangle.set_facecolors("none")
    #patch_collection_triangle.set_antialiaseds(False)    # Blends things a little
    #patch_collection_triangle.set_cmap(color_map)        # Select our color_map
    patch_collection_triangle.set_snap(None)

    # Now apply the patch_collection to our axis (ie plot it)
    ax.add_collection(patch_collection_triangle)


    #patch_collection_cell.set_array(lat[:])
    patch_collection_cell.set_linewidths(0.2)
    patch_collection_cell.set_linestyle("-")
    patch_collection_cell.set_edgecolors("black")         # No Edge Colors
    patch_collection_cell.set_facecolors("none")
    #patch_collection_cell.set_antialiaseds(False)    # Blends things a little
    #patch_collection_cell.set_cmap(color_map)        # Select our color_map
    patch_collection_cell.set_snap(None)

    # Now apply the patch_collection to our axis (ie plot it)
    ax.add_collection(patch_collection_cell)


    # Convert the lat-lon coordinates to basemap xy coordinates for plotting
    lons, lats = bmap(varlons, varlats) #, marker = '.', color='r')

    bmap.scatter(lons, lats, s=0.1, marker = 'x', color='r', zorder=5)

    # Create the title as you see fit
    plt.title(variable)
    plt.style.use(style) # Set the style that we choose above

    plt.savefig(out_dir+"/"+variable+'.png', dpi=500)

    #plt.close(fig)
    plt.show()

    #patch_collection_triangle.remove()
    #patch_collection_cell.remove()
    return

Let's test this. First we need to read the mesh file and extract a variable of interest. This will extract (lon,lat) of the geometric variable you wish to plot, for instance cell centers, edge midpoints, etc.

In [ ]:
#'''Geometric variable you want to plot from that file'''
variable = "Edge"

# '''File you want to plot from'''
#file = "grid/grid_mpas.nc"
file = out_basepath+"_mpas.nc"

plot_geometry(variable, file)


-----

## Uniform grid construction

Jigsaw creates spherical grids based on latitute-longitude definitions of the resolutions we want in each part of the globe. We will start with an uniform resolution grid, but the main code to call Jigsaw is the following.

> Important: This will not create a grid based on icosahedron, but will directly using the underlying grid optimization method from Jigsaw to distribute points on the sphere.



In [ ]:

def jigsaw_gen_sph_grid(cellWidth, x, y, earth_radius=6371.0e3, basename="mesh" ):

    """
    A function for building a jigsaw spherical mesh
    Parameters
    ----------
    cellWidth : ndarray
        The size of each cell in the resulting mesh as a function of space
    x, y : ndarray
        The x and y coordinates of each point in the cellWidth array (lon and
        lat for spherical mesh)
    on_sphere : logical, optional
        Whether this mesh is spherical or planar
    earth_radius : float, optional
        Earth radius in meters
    """
    # Authors
    # -------
    #by P. Peixoto in Dec 2021
    # based on MPAS-Tools file from Mark Petersen, Phillip Wolfram, Xylar Asay-Davis


    # setup files for JIGSAW
    opts = jig.jigsaw_jig_t()
    opts.geom_file = basename+'.msh'
    opts.jcfg_file = basename+'.jig'
    opts.mesh_file = basename+'-MESH.msh'
    opts.hfun_file = basename+'-HFUN.msh'

    # save HFUN data to file
    hmat = jig.jigsaw_msh_t()

    hmat.mshID = 'ELLIPSOID-GRID'
    hmat.xgrid = np.radians(x)
    hmat.ygrid = np.radians(y)
    hmat.value = cellWidth
    jig.savemsh(opts.hfun_file, hmat)

    # define JIGSAW geometry
    geom = jig.jigsaw_msh_t()
    geom.mshID = 'ELLIPSOID-MESH'
    geom.radii = earth_radius*1e-3*np.ones(3, float)
    jig.savemsh(opts.geom_file, geom)

    # build mesh via JIGSAW!
    opts.hfun_scal = 'absolute'
    opts.hfun_hmax = float("inf")
    opts.hfun_hmin = 0.0
    opts.mesh_dims = +2  # 2-dim. simplexes
    opts.mesh_iter = 500000
    opts.optm_qlim = 0.9375
    opts.optm_qtol = 1.0e-6
    opts.optm_iter = 500000
    opts.verbosity = +1
    jig.savejig(opts.jcfg_file, opts)

    #Call jigsaw
    process = subprocess.call(['jigsaw', opts.jcfg_file])

    return opts.mesh_file


The user specification step is to define a function that creates a high resolution lat-lon grid with the wanted resolution at each lat-lon point. Here, this resolution will be assumed constant.

In [ ]:

def cellWidthVsLatLon(r=70):
    """
    Create cell width array for this mesh on a regular latitude-longitude grid.

    Input
    ---------
    r : float
        constant desired cell width resolution in km

    Returns
    -------
    cellWidth : ndarray
        m x n array of cell width in km
    lon : ndarray
        longitude in degrees (length n and between -180 and 180)
    lat : ndarray
        longitude in degrees (length m and between -90 and 90)
    """
    dlat = r/1000 #Make the lat-lon grid ~ 10x finer than resolution at equator, where 1deg ~ 100km
    dlon = dlat
    constantCellWidth = r  #in km

    nlat = int(180./dlat) + 1
    nlon = int(360./dlon) + 1

    lat = np.linspace(-90., 90., nlat)
    lon = np.linspace(-180., 180., nlon)

    cellWidth = constantCellWidth * np.ones((lat.size, lon.size))

    return cellWidth, lon, lat


Let's build the mesh now

In [ ]:

# Set a basename for output of grid files
output = "grid_unif"

out_dir=os.path.abspath(output)
out_base=os.path.basename(output)
out_basepath=out_dir+"/"+out_base
out_filename=out_dir+"/"+out_base+"_mpas.nc"

if not os.path.exists(out_dir):
    os.makedirs(out_dir)
else:
    print("Base dir already exists: ", out_dir)


# Grid options

#Basic resolution (in km)
r = 300

#Density based grids
#------------------
# Uniform resolution spherical grid (hand tune cellWidthVsLatLon function)
#                 Provide also a resolution in km
cellWidth, lon, lat = cellWidthVsLatLon(r)

#Locally refined grid
#Variable resolution spherical grid (hand tune localrefVsLatLon function)
#                 Provide also a resolution with in km
#cellWidth, lon, lat = localrefVsLatLon(r, p=p)

# Generate grid
mesh_file = jigsaw_gen_sph_grid(cellWidth, lon, lat, basename=out_basepath)

#Icosahedral grid
#mesh_file = jutil.jigsaw_gen_icos_grid(basename=out_basepath, level=4)

#Convert jigsaw mesh to netcdf
jigsaw_to_netcdf(msh_filename=mesh_file, output_name=out_basepath+'_triangles.nc', on_sphere=True,
                     sphere_radius=1.0)

#convert to mpas grid specific format
write_netcdf( convert(xarray.open_dataset(out_basepath+'_triangles.nc'),
        dir=out_dir, graphInfoFileName=out_basepath+"_graph.info"),
        out_filename)



Let's go straight to a plot of the generated grid.

In [ ]:
#'''Geometric variable you want to plot from that file'''
variable = "Edge"

# '''File you want to plot from'''
#file = "grid/grid_mpas.nc"
file = out_basepath+"_mpas.nc"

plot_geometry(variable, file)

## Variable resolution grid

For variable resolution grids, we need to create a function that tells us what resolution we want in which region.

In [ ]:


def localrefVsLatLon(r=30, l=350, radius_low=100, transition_radius=600, p=False):
    """
    Create cell width array for this mesh on a locally refined latitude-longitude grid.
    Input
    ---------
    h : float
        grid spacing for high resolution area in km
    l : float
        grid spacing for low resolution area in km
    radius_low : float
        radius of influence of low resolution area in km
    transition_radius : float
        radius of the transition zone between high and low resolution in km

    Returns
    -------
    cellWidth : ndarray
        m x n array of cell width in km
    lon : ndarray
        longitude in degrees (length n and between -180 and 180)
    lat : ndarray
        longitude in degrees (length m and between -90 and 90)
    """

    dlat = 0.5 #Make the lat-lon grid ~ 10x finer than resolution at equator, where 1deg ~ 100km
    dlon = dlat
    constantCellWidth = r  #in km
    print("Trying to set grid spacing of high resolution zone to approximately: "+str(constantCellWidth))

    nlat = int(180./dlat) + 1
    nlon = int(360./dlon) + 1

    lat = np.linspace(-90., 90., nlat)
    lon = np.linspace(-180., 180., nlon)

    lons, lats = np.meshgrid(lon, lat)
    #Calculate distances to center (lat=0,lon=0)
    centre_lon = -40
    centre_lat = -20
    dists = latlon_to_distance_center(lons, lats, centre_lon, centre_lat)

    if p:
        print("Centre point and distances to/from this point")
        h = plt.contourf(lons, lats, dists)
        plt.axis('scaled')
        plt.colorbar()
        plt.show()

    #Parameters
    #------------------------------

    # Radius (in km) of high resolution area
    maxdist = radius_low
    print("Radius of high resolution area set approximately to: "+str(maxdist))

    distance = transition_radius/10
    print("Transition zone from high to low resolution set approximately to: "+  str(transition_radius))
    # (increase_of_resolution) / (distance)
    slope = 10./distance

    # Gammas
    gammas = l
    print("Global grid spacing set to approximately: "+str(gammas))

    # distance (in km) of transition zone belt: ratio / slope
    maxepsilons = 10000.
    epsilons = gammas/slope

    if(epsilons > maxepsilons):
        print("Transition zone too wide: set to 10,000 km")
        epsilons = maxepsilons

    # ## If radius of transition zone is not provided, try to find best value
    # if not(transition_radius):
    #     # distance (in km) of transition zone belt: ratio / slope
    #     maxepsilons = 10000.
    #     epsilons = gammas/slope

    #     if(epsilons > maxepsilons):
    #         epsilons = maxepsilons
    #     print("Transition zone radius not provided. Value set to: "+str(epsilons))
    # else:
    #     epsilons = transition_radius
    #     print("Transition zone radius provided: "+str(epsilons))


    # initialize with resolution = r (min resolution)
    resolution = constantCellWidth * np.ones(np.shape(dists))

    # point in transition zone
    transition_zone = (dists > maxdist) & (dists <= maxdist + epsilons)
    sx = (dists - maxdist ) * slope
    transition_values = constantCellWidth + sx
    resolution = np.where(transition_zone, transition_values, resolution)

    # further points
    far_from_center = (dists > maxdist + epsilons)
    resolution[far_from_center] += epsilons * slope

    if p:
        print("Target resolution for grid:")
        h = plt.contourf(lons, lats, resolution, cmap="viridis", levels=100)
        plt.axis('scaled')
        plt.colorbar()
        plt.show()

    print(np.min(resolution), np.max(resolution))

    cellWidth = resolution #constantCellWidth * np.ones((lat.size, lon.size))

    return cellWidth, lon, lat

def density_function_dists(dists, slope=None, gammas=None, maxdist=None,
                           maxepsilons=None):

    epsilons = gammas/slope
    if epsilons > maxepsilons:
        epsilons = maxepsilons

    # initialize with resolution = 1
    resolution = np.ones(np.shape(dists))

    # point in transition zone
    transition_zone = (dists > maxdist) & (dists <= maxdist + epsilons)
    sx = (dists -maxdist ) *slope
    transition_values = 1.0 + sx
    resolution = np.where(transition_zone, transition_values, resolution)

    # further points
    far_from_center = (dists > maxdist + epsilons)
    resolution[far_from_center] += epsilons * slope

    # convert to density
    dens_f = 1 / resolution**2
    return dens_f


def latlon_to_distance_center(lon, lat, lon0, lat0):
    earth_radius = 6371.0e3

    lon, lat = map(np.radians, [lon, lat])
    lon0, lat0 = map(np.radians, [lon0, lat0])

    haver_formula = np.sin((lat-lat0) / 2.0) ** 2 + \
                    np.cos(lat0)*np.cos(lat) * np.sin((lon-lon0) / 2.0) ** 2

    dists = 2 * np.arcsin(np.sqrt(haver_formula)) * earth_radius/1000
    return dists



In [ ]:

# Set a basename for output of grid files
output = "grid_var_res"

out_dir=os.path.abspath(output)
out_base=os.path.basename(output)
out_basepath=out_dir+"/"+out_base
out_filename=out_dir+"/"+out_base+"_mpas.nc"

if not os.path.exists(out_dir):
    os.makedirs(out_dir)
else:
    print("Base dir already exists: ", out_dir)


# Grid options

#Basic resolution (in km)
r = 300

#Density based grids
#------------------
# Uniform resolution spherical grid (hand tune cellWidthVsLatLon function)
#                 Provide also a resolution in km
#cellWidth, lon, lat = cellWidthVsLatLon(r)

#Locally refined grid
#Variable resolution spherical grid (hand tune localrefVsLatLon function)
#                 Provide also a resolution with in km
cellWidth, lon, lat = localrefVsLatLon(r=60, l=350, radius_low=1000, transition_radius=1500, p=True)


# Generate grid
mesh_file = jigsaw_gen_sph_grid(cellWidth, lon, lat, basename=out_basepath)

#Icosahedral grid
#mesh_file = jutil.jigsaw_gen_icos_grid(basename=out_basepath, level=4)

#Convert jigsaw mesh to netcdf
jigsaw_to_netcdf(msh_filename=mesh_file, output_name=out_basepath+'_triangles.nc', on_sphere=True,
                     sphere_radius=1.0)

#convert to mpas grid specific format
write_netcdf( convert(xarray.open_dataset(out_basepath+'_triangles.nc'),
        dir=out_dir, graphInfoFileName=out_basepath+"_graph.info"),
        out_filename)


In [ ]:
#'''Geometric variable you want to plot from that file'''
variable = "Edge"

# '''File you want to plot from'''
#file = "grid/grid_mpas.nc"
file = out_basepath+"_mpas.nc"

plot_geometry(variable, file)

## ❗Exercise ❗

Generate a grid with coarse global resolution of approximately 300km with local refinement around a region that covers the whole South America Continent, with gradual transition towards a 30km grid in the continent.

Plot the final grid.